Funcion para levantar diccionario de peliculas

In [1]:
import requests
import json

def extract():
    path="./"
    f = open(path + "tmdb.json")
    if f:
        return json.loads(f.read());

Funcion para cargar indice en elastic

In [2]:
def reindex(analysisSettings={}, mappingSettings={}, movieDict={}):
    ELASTIC_URL = "http://localhost:9200"
    TMPDB_URLBASE = ELASTIC_URL + "/tmpdb"
    settings = {
        "settings": {
            "number_of_shards": 1,
            "index" : {
                "analysis" : analysisSettings,
            }
        }
    }
    
    if mappingSettings:
        settings["mappings"] = mappingSettings
        
    resp = requests.delete(TMPDB_URLBASE)
    resp = requests.put(TMPDB_URLBASE, data=json.dumps(settings))
    
    bulkMovies = ""
    for id, movie in movieDict.items():
        addCmd = {
            "index": {
                "_index": "tmpdb",
                "_type": "movie",
                "_id": movie["id"],
                
            }
        }
        
        bulkMovies += json.dumps(addCmd) + "\n" + json.dumps(movie) + "\n"
    resp = requests.post(ELASTIC_URL + "/_bulk", data=bulkMovies)
        

Cargo peliculas

In [3]:
movieDict = extract()
reindex(movieDict=movieDict)